### Import models and load data

In [1]:
from torchvision import datasets
import torch
import dlc_practical_prologue as prologue
import matplotlib.pyplot as plt

# Import all models
from BaseNet import *
from ConvNet1 import *
#from NetSharing import *

mini_batch_size = 1000
nb_epochs = 300
nb_runs = 2
eta = 0.001

train_input, train_target, train_classes, test_input, test_target, test_classes = \
    prologue.generate_pair_sets(nb=1000)

C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\DanielSmarda\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### 1channel2images

In [2]:
from importlib import reload
import _1channel2images
reload(_1channel2images)
from _1channel2images import *

print("Working with 1channel2images framework, nb_classes = ", nb_classes)

#model = BaseNet1C(nb_classes)
model_1C = ConvNet1_1C(nb_classes)
optimizer_1C = torch.optim.SGD(model_1C.parameters(), lr=eta, momentum=0.95)
test_results_1C = multiple_training_runs(model_1C, 2, optimizer_1C, train_input, train_classes,
                                      test_input, test_target, test_classes, mini_batch_size, nb_epochs)
write_to_csv('1channel2images.csv', model_1C, test_results_1C)

Working with 1channel2images framework, nb_classes =  10


KeyboardInterrupt: 

In [ ]:
# This is just for visualization of our results but it will have to be taken away for the report 
# since we can't use any additional libraries
import pandas as pd

print(test_results_1C)
write_to_csv('1channel2images.csv', model_1C, test_results_1C)
data = pd.read_csv('1channel2images.csv')
data.head()

### 2channels1image

In [ ]:
import _2channels1image
reload(_2channels1image)
from _2channels1image import *

print("Working with 2channels1image framework, nb_classes = ", nb_classes)

model_2C = ConvNet1_2C(nb_classes)
optimizer_2C = torch.optim.SGD(model_2C.parameters(), lr=0.00001, momentum=0.95)  
test_results_2C = multiple_training_runs(model_2C, 2, optimizer_2C, train_input, train_target,
                           test_input, test_target, mini_batch_size, nb_epochs)
write_to_csv('2channels1image.csv', model_2C, test_results_2C)

In [ ]:
print(test_results_2C)
write_to_csv('2channels1image.csv', model_2C, test_results_2C)
data = pd.read_csv('2channels1image.csv')
data.head()

### Weight Sharing

In [ ]:
import weight_sharing
reload(weight_sharing)
from weight_sharing import *

print("Working with weight_sharing framework")

model_ws = NetSharing1()
optimizer_ws = torch.optim.SGD(model_ws.parameters(), lr=0.00001, momentum=0.95)  
test_results_ws = multiple_training_runs(model_ws, nb_runs, optimizer_ws, train_input, train_target,
                           test_input, test_target, mini_batch_size, nb_epochs)
write_to_csv('weightsharing.csv', model_ws, test_results_ws)

In [ ]:
print(test_results_ws)
write_to_csv('weightsharing.csv', model_ws, test_results_ws)
data = pd.read_csv('weightsharing.csv')
data.head()

### Auxiliary Losses

In [ ]:
import auxiliary_losses
reload(auxiliary_losses)
from auxiliary_losses import *
import Incept1
reload(Incept1)
from Incept1 import *
nb_epochs = 300
print("Working with auxiliary losses framework")

model_aux = Incept1()
optimizer_aux = torch.optim.SGD(model_aux.parameters(), lr=0.00001, momentum=0.95)  
test_results_aux = multiple_training_runs(model_aux, nb_runs, optimizer_aux, train_input, train_target, train_classes, 
                                          test_input, test_target, test_classes, mini_batch_size, nb_epochs)
write_to_csv('auxiliary_losses.csv', model_aux, test_results_aux)

In [ ]:
print(test_results_aux)
write_to_csv('auxiliary_losses.csv', model_aux, test_results_aux)
data = pd.read_csv('auxiliary_losses.csv')
data.head()

## Weight Sharing and Auxiliary Loss Models(Youssef)

In [8]:
print(train_input.shape, train_target.shape, train_classes.shape, test_input.shape, test_target.shape, test_classes.shape)

torch.Size([1000, 2, 14, 14]) torch.Size([1000]) torch.Size([1000, 2]) torch.Size([1000, 2, 14, 14]) torch.Size([1000]) torch.Size([1000, 2])


In [6]:
from torch.autograd import Variable
from WSharingModel import *
from torch import nn, optim
from torch.nn import CrossEntropyLoss
from torch.nn import BCEWithLogitsLoss
import torch
from torchvision import datasets
from torch import optim
import dlc_practical_prologue as prologue
from AuxModel import *
from torch.autograd import Variable
from torch.nn import functional as F
from time import time
from SimpleCNN import *


#### Function that draws the evolution of the loss during training.

In [7]:
def visualize_loss(model,loss_model,lr):
    plt.plot(loss_model[0],loss_model[1], color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Evolution of the loss during training with learning rate {:>5} of the model {:>5}.'.format(lr,model.__class__.__name__))
    plt.show()

#### Function that prints the results of the prediction one the model is trained.

In [8]:
def print_results(model,optimizer,learning_rate,NB_EPOCHS,MINI_BATCH_SIZE, train_input, train_target,test_input, test_target, time_diff):
     print('model: {:6}, optimizer: {:6}, learning rate: {:6}, num epochs: {:3}, '
                    'mini batch size: {:3}, train error: {:5.2f}%, test error: {:5.2f}%, time: {:.3f}'.format(
                    model.__class__.__name__,
                    optimizer.__name__,
                    learning_rate,
                    NB_EPOCHS,
                    MINI_BATCH_SIZE,
                    model.compute_nb_errors(model, train_input, train_target, MINI_BATCH_SIZE) / train_input.size(0) * 100,
                    model.compute_nb_errors(model, test_input, test_target, MINI_BATCH_SIZE) / test_input.size(0) * 100,
                    time_diff
                    )
                )

### Treatement of our Training and Testing Data.

In [9]:
# normalize it
mean, std = train_input.mean(), train_input.std() 
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

train_input, train_target, train_classes = Variable(train_input), Variable(train_target), Variable(train_classes)
test_input, test_target = Variable(test_input), Variable(test_target)

### We define our Learning parameters.

In [11]:
NB_EPOCHS = 25
MINI_BATCH_SIZE = 100
learning_rates = [0.1, 0.01]

### We define our optimizers and losses and weights for the auxiliary losses.

In [12]:
op = torch.optim.SGD
losses = [CrossEntropyLoss(),BCEWithLogitsLoss()]
alpha = 0.5
beta = 1

### Training of Different Models and Results on Training and Testing set.

In [19]:
 def calculate_results(result_dct):
    k = 0
    for j in range(4,5):
        for i in range(len(learning_rates)):
            models = [WSModel(),WSModel1(),AuxModel(),AuxModel1(), SimpleModel()]
            model = models[j]
            optimizer = op(model.parameters(),lr = learning_rates[i])
            if (j<2):
                #print("here")
                start_time = time()
                loss_aux = train_model_WS(model, optimizer,  train_input, train_target, NB_EPOCHS, MINI_BATCH_SIZE,losses[0])
                end_time = time()
            elif (j==4):
                start_time = time()
                loss_aux = train_model(model, optimizer,  train_input, train_target, NB_EPOCHS, MINI_BATCH_SIZE,losses[0])
                end_time = time()
            elif(j>1):
                #print("here here")
                start_time = time()
                loss_aux = train_model_AM(model, optimizer,  train_input, train_target, train_classes,NB_EPOCHS, MINI_BATCH_SIZE,losses[0],alpha,beta)
                end_time = time()
            #print("model:",model)
            time_diff = end_time - start_time
            print_results(model,op,learning_rates[i],NB_EPOCHS,MINI_BATCH_SIZE, train_input, train_target,test_input, test_target, time_diff)
            print(result_dct[k])
            train_error = model.compute_nb_errors(model, train_input, train_target, MINI_BATCH_SIZE) / train_input.size(0) * 100
            test_error = model.compute_nb_errors(model, test_input, test_target, MINI_BATCH_SIZE) / test_input.size(0) * 100
            result_dct[k]['train_error'].append(train_error)
            result_dct[k]['test_error'].append(test_error)
            result_dct[k]['time'].append(time_diff)
            k += 1
    return result_dct
            #visualize_loss(model,loss_aux,learning_rates[i])

In [18]:
from collections import defaultdict
result_dct = defaultdict(dict)
for i in range(10):
    result_dct[i]['train_error'] = []
    result_dct[i]['test_error'] = []
    result_dct[i]['time'] = []
for i in range(10):
    
    train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(nb=1000)
    # normalize it
    mean, std = train_input.mean(), train_input.std() 
    train_input.sub_(mean).div_(std)
    test_input.sub_(mean).div_(std)

    train_input, train_target, train_classes = Variable(train_input), Variable(train_target), Variable(train_classes)
    test_input, test_target = Variable(test_input), Variable(test_target)
    
    print(result_dct)
    result_dct = calculate_results(result_dct)

defaultdict(<class 'dict'>, {0: {'train_error': [], 'test_error': [], 'time': []}, 1: {'train_error': [], 'test_error': [], 'time': []}, 2: {'train_error': [], 'test_error': [], 'time': []}, 3: {'train_error': [], 'test_error': [], 'time': []}, 4: {'train_error': [], 'test_error': [], 'time': []}, 5: {'train_error': [], 'test_error': [], 'time': []}, 6: {'train_error': [], 'test_error': [], 'time': []}, 7: {'train_error': [], 'test_error': [], 'time': []}, 8: {'train_error': [], 'test_error': [], 'time': []}, 9: {'train_error': [], 'test_error': [], 'time': []}})
Loss at epoch  0 : 0.6727057099342346  
Loss at epoch 24 : 0.08629505336284637  
model: SimpleModel, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 100, train error:  1.30%, test error: 21.30%, time: 21.226
{'train_error': [], 'test_error': [], 'time': []}
Loss at epoch  0 : 0.6985001564025879  
Loss at epoch 24 : 0.556518018245697  
model: SimpleModel, optimizer: SGD   , learning rate:   0.01, num

Loss at epoch 24 : 0.06890102475881577  
model: SimpleModel, optimizer: SGD   , learning rate:    0.1, num epochs:  25, mini batch size: 100, train error:  1.40%, test error: 17.50%, time: 20.447
{'train_error': [1.3, 0.0, 0.6, 0.3, 0.0], 'test_error': [21.3, 16.900000000000002, 18.4, 18.6, 17.4], 'time': [21.22620129585266, 29.01410484313965, 17.910000801086426, 28.94403076171875, 21.57417583465576]}
Loss at epoch  0 : 0.6777381300926208  
Loss at epoch 24 : 0.5597725510597229  
model: SimpleModel, optimizer: SGD   , learning rate:   0.01, num epochs:  25, mini batch size: 100, train error: 28.70%, test error: 26.90%, time: 20.527
{'train_error': [21.099999999999998, 23.7, 22.3, 23.200000000000003, 21.7], 'test_error': [24.8, 26.700000000000003, 25.1, 26.400000000000002, 24.0], 'time': [20.700235605239868, 26.889419078826904, 16.40429139137268, 19.672778129577637, 18.92291283607483]}
defaultdict(<class 'dict'>, {0: {'train_error': [1.3, 0.0, 0.6, 0.3, 0.0, 1.4000000000000001], 'test_e

defaultdict(<class 'dict'>, {0: {'train_error': [1.3, 0.0, 0.6, 0.3, 0.0, 1.4000000000000001, 0.1, 0.0, 0.0], 'test_error': [21.3, 16.900000000000002, 18.4, 18.6, 17.4, 17.5, 19.3, 15.1, 15.4], 'time': [21.22620129585266, 29.01410484313965, 17.910000801086426, 28.94403076171875, 21.57417583465576, 20.447258949279785, 19.211263418197632, 19.755135536193848, 22.004987478256226]}, 1: {'train_error': [21.099999999999998, 23.7, 22.3, 23.200000000000003, 21.7, 28.7, 24.6, 26.1, 24.5], 'test_error': [24.8, 26.700000000000003, 25.1, 26.400000000000002, 24.0, 26.900000000000002, 25.8, 26.3, 24.099999999999998], 'time': [20.700235605239868, 26.889419078826904, 16.40429139137268, 19.672778129577637, 18.92291283607483, 20.526583909988403, 19.0296413898468, 20.30643630027771, 20.16654896736145]}, 2: {'train_error': [], 'test_error': [], 'time': []}, 3: {'train_error': [], 'test_error': [], 'time': []}, 4: {'train_error': [], 'test_error': [], 'time': []}, 5: {'train_error': [], 'test_error': [], 't

In [20]:
import statistics
for k,v in result_dct.items():
    print('mean of train_error of model {} = {}'.format(k, statistics.mean(v['train_error'])))
    print('stdev of train_error of model {} = {}'.format(k, statistics.stdev(v['train_error'])))
    print('mean of test_error of model {} = {}'.format(k, statistics.mean(v['test_error'])))
    print('stdev of test_error of model {} = {}'.format(k, statistics.stdev(v['test_error'])))
    print('mean of time of model {} = {}'.format(k, statistics.mean(v['time'])))
    print('stdev of time of model {} = {}'.format(k, statistics.stdev(v['time'])))
    print()

mean of train_error of model 0 = 0.76
stdev of train_error of model 0 = 1.2267391282945566
mean of test_error of model 0 = 18.1
stdev of test_error of model 0 = 2.097617696340303
mean of time of model 0 = 21.828443098068238
stdev of time of model 0 = 4.004432737783873

mean of train_error of model 1 = 24.0
stdev of train_error of model 1 = 2.222111108333195
mean of test_error of model 1 = 25.95
stdev of test_error of model 1 = 1.5980891367157495
mean of time of model 1 = 20.12246639728546
stdev of time of model 1 = 2.6933051255235134



StatisticsError: mean requires at least one data point

TypeError: can't pickle torch._C.Generator objects